<a href="https://colab.research.google.com/github/buingohoanglong/SVM-kNN-DecisionTree/blob/main/SVM_kNN_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Load data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [220]:
!ls

drive  sample_data


In [221]:
import pandas as pd

In [222]:
dataset = pd.read_csv('drive/MyDrive/desktop/ADA/extra/spam_ham_dataset.csv')

In [223]:
dataset

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [224]:
data_X = dataset['text']
data_y = dataset['label_num']

In [ ]:
data_X

In [ ]:
data_y

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [226]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [227]:
def text_cleaner(text):
    text = text.lower() # lower case text
    text = re.sub(r"'s\b","",text)  # remove 's at the end of each word
    # remove punctuations
    text = re.sub("[^a-zA-Z]", " ", text) # how about numbers ???
    return text

In [228]:
def preprocess(sentence):
  sentence = text_cleaner(sentence)
  tokens = word_tokenize(sentence)
  stop_words = set(stopwords.words('english'))
  tokens = [tk for tk in tokens if not tk in stop_words]
  return tokens

In [229]:
sentences = [preprocess(s) for s in data_X]

In [230]:
labels = [l if l == 1 else -1 for l in data_y]

# II. Load google pretrained word2vec

In [13]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [14]:
print(path)

/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [15]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
word_vectors['facebook'].shape

# III. Prepare data

In [235]:
import numpy as np

In [236]:
EMBEDDING_DIM = 300

In [237]:
X = np.zeros((len(sentences),EMBEDDING_DIM))

In [238]:
index = 0
for sentence in sentences:
  for token in sentence:
    try:
      embedding_vector = word_vectors[token]
      X[index] += embedding_vector
    except KeyError:
      X[index] += np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)
  index += 1

In [ ]:
print(X[0])

In [239]:
from sklearn.model_selection import train_test_split

In [248]:
X_train, X_test, y_train, y_test = train_test_split(X, np.array(labels))
X_train = X_train.T
X_test = X_test.T
y_train = y_train.reshape(1, y_train.shape[0])
y_test = y_test.reshape(1, y_test.shape[0])

In [249]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(300, 3878)
(1, 3878)
(300, 1293)
(1, 1293)


# IV. Soft Margin SVM model

In [250]:
class SVM:
  def __init__(self):
    self.weights = None

  def calculate_accurracy(self, predicts, y):
    f = np.frompyfunc(lambda x: x if x == 1 else 0,1,1)
    num_correct_predicts = np.sum(f(predicts*y))
    num_predicts = y.shape[1]
    acc = num_correct_predicts / num_predicts
    return acc

  def has_positive_hinge_loss(self, w_bar, x_bar, y):
    f = np.frompyfunc(lambda x: 1 if x > 0 else 0, 1, 1)
    return f(1 - y*w_bar.T.dot(x_bar))

  def fit(self, X, y, learning_rate=0.001, ld=1, loops=1000):
    X_bar = np.concatenate((X, np.ones((1, X.shape[1]))), axis=0) # extend
    self.weights = np.ones((X_bar.shape[0], 1)) # extend

    for i in range(loops):
      if i%100 == 0:
        predicts, acc = self.predict(X,y)
        print(f"Acc: {acc}")
        
      temp = self.has_positive_hinge_loss(self.weights, X_bar, y)
      regularization_gradient = ld * np.concatenate((self.weights[:-1], np.array([[0]])), axis=0)
      hinge_gradient = np.sum(-temp*y*X_bar, axis=1).reshape(X_bar.shape[0], 1)
      self.weights = self.weights - learning_rate * ( hinge_gradient + regularization_gradient )


  def predict(self, X, y=None):
    X_bar = np.concatenate((X, np.ones((1, X.shape[1]))), axis=0) # extend
    predicts = np.sign(self.weights.T.dot(X_bar))

    if y is not None: # used for test purpose
      # calculate accurracy
      acc = self.calculate_accurracy(predicts, y)
      return predicts, acc
    else:
      return predicts

  def save(self, file_name):
    with open(file_name, 'wb') as f:
      np.save(f, self.weights)

  def load(self, file_name):
    with open(file_name, 'rb') as f:
      self.weights = np.load(f, allow_pickle=True)
  

In [251]:
svm_model = SVM()
svm_model.fit(X_train, y_train)

Acc: 0.6428571428571429
Acc: 0.9339865910263022
Acc: 0.9381124290871583
Acc: 0.943011861784425
Acc: 0.9489427539969056
Acc: 0.9507478081485302
Acc: 0.9507478081485302
Acc: 0.9535843218153688
Acc: 0.9476534296028881
Acc: 0.9530685920577617


In [252]:
svm_model.save('drive/MyDrive/desktop/ADA/extra/weights.npy')

In [253]:
svm_model.load('drive/MyDrive/desktop/ADA/extra/weights.npy')

In [254]:
predicts, acc = svm_model.predict(X_test, y_test)

In [255]:
acc

0.9234338747099768

# V. kNN model

In [256]:
class kNN:
  def __init__(self):
    self.X_train = None
    self.y_train = None

  def calculate_accurracy(self, predicts, y):
    f = np.frompyfunc(lambda x: x if x == 1 else 0,1,1)
    num_correct_predicts = np.sum(f(predicts*y))
    num_predicts = y.shape[1]
    acc = num_correct_predicts / num_predicts
    return acc
  
  def euclidean_distance(self, x, X):
    return np.sum((x - X)**2, axis=0)

  def voting(self, k_nearest, distances, labels):
    scores = np.zeros((2,1))
    for neighbour in k_nearest:
      d = distances[neighbour]
      if d == 0:
        return labels[:,neighbour]

      if labels[:,neighbour] == 1:
        scores[0] += 1/d
      else:
        scores[1] += 1/d

    return 1 if scores[0] > scores[1] else -1

  def fit(self, X, y):
    self.X = X
    self.y = y

  def predict(self, X_test, y_test=None, k=10):
    predicts = np.zeros((1, y_test.shape[1]))
    for i in range(X_test.shape[1]):
      x = X_test[:,i].reshape((X_test.shape[0], 1))
      distances = self.euclidean_distance(x, self.X)
      k_nearest = np.argpartition(distances, k)
      predicts[:,i] = self.voting(k_nearest, distances, self.y)

    if y_test is not None:  # used for test purpose
      acc = self.calculate_accurracy(predicts, y_test)
      return predicts, acc
    else:
      return predicts
  

In [257]:
knn_model = kNN()
knn_model.fit(X_train, y_train)
predicts, acc = knn_model.predict(X_test, y_test)

In [258]:
acc

0.7138437741686001

# VI. Decision Tree model

In [259]:
class Node:
  def __init__(self, ids, depth):
    self.ids = ids # index of datapoint in dataset
    self.entropy = None

    # leaf node attribute
    self.label = None

    # internal node attribute
    self.split_attribute = None # index of attribute to split at this node
    self.split_value = None # value to be split at self.slit_attribute
    self.children = []  # list of child nodes (left and right)

    self.depth = depth

  def set_state(self, split_attribute, split_value, children):
    self.split_attribute = split_attribute
    self.split_value = split_value
    self.children = children

In [260]:
from functools import *
import math

class DecisionTree:
  def __init__(self, max_depth):
    self.X = None # list of data point, each data point is a list of attributes (write in row)
    self.y = None  # list of labels

    self.root = None

    self.max_depth = max_depth


  def entropy(self, node):
    labels, counts = np.unique(self.y[:,node.ids], return_counts=True)
    probs = counts / node.ids.shape[0]
    node.entropy = 0 if np.any(probs == 0) else -np.sum(probs*np.log(probs))
    return node.entropy


  def set_label(self, node):
    labels, counts = np.unique(self.y[:,node.ids], return_counts=True)
    node.label = labels[counts.argmax()]


  def split(self, node, attribute):
    split_value = None
    best_information_gain = 0
    children = []

    X_concat = np.concatenate((self.X[node.ids],self.y[:,node.ids].T,node.ids.reshape((1,-1)).T), axis=1)
    X_concat_sorted = X_concat[np.argsort(X_concat[:, attribute])]

    split_value_set = np.unique(X_concat_sorted[:,attribute])
    for value in split_value_set:
      i = X_concat_sorted[:,attribute].searchsorted(value)
      left_ids, right_ids = X_concat_sorted[:i,-1].astype(int), X_concat_sorted[i:,-1].astype(int)
      left_child = Node(ids=left_ids, depth=node.depth+1)
      right_child = Node(ids=right_ids, depth=node.depth+1)

      if left_child.ids.shape[0] == node.ids.shape[0] or right_child.ids.shape[0] == node.ids.shape[0]:
        information_gain = 0
      else:
        information_gain = node.entropy - (left_child.ids.shape[0]*self.entropy(left_child) + right_child.ids.shape[0]*self.entropy(right_child)) / node.ids.shape[0]

      if information_gain > best_information_gain:
        best_information_gain = information_gain
        children = [left_child, right_child]
        split_value = value

    return best_information_gain, attribute, split_value, children



  def split_node(self, node):
    best_information_gain = 0
    split_attribute = None
    split_value = None
    children = []

    for attribute in range(self.X.shape[1]):
      information_gain, current_split_attribute, current_split_value, current_children = self.split(node, attribute)
      if information_gain > best_information_gain:
        best_information_gain = information_gain
        split_attribute = current_split_attribute
        split_value = current_split_value
        children = current_children

    node.set_state(split_attribute, split_value, children)


  def build_tree(self):
    # build tree in bfs manner
    queue = [self.root]
    while len(queue) != 0:
      node = queue.pop(0)
      if self.entropy(node) == 0 or node.depth >= self.max_depth: # leaf node
        self.set_label(node)
      else: # internal node
        self.split_node(node)
        if node.children == []:
          self.set_label(node)
        else:
          for child in node.children:
            queue.append(child)


  def fit(self, X, y):
    self.X = X.T # (num data, num attr)
    self.y = y  # (1, num data)

    print(f"X shape: {self.X.shape}")
    print(f"y shape: {self.y.shape}")

    ids = np.arange(self.y.shape[1], dtype=int)
    print(f"ids shape: {ids.shape}")
    self.root = Node(ids=ids, depth=0)
    
    self.build_tree()


  def __predict(self, x, node):
    if node.label is not None:
      return node.label

    if x[node.split_attribute] >= node.split_value:
      return self.__predict(x, node.children[1])
    else:
      return self.__predict(x, node.children[0])


  def predict(self, x):
    return self.__predict(x, self.root)

  def test(self, X, y):
    X_test = X.T
    predicts = np.zeros(y.shape)

    for index in range(X_test.shape[0]):
      x = X_test[index]
      predicts[0, index] = self.predict(x)

    # calculate accurracy
    f = np.frompyfunc(lambda x: x if x == 1 else 0,1,1)
    num_correct_predicts = np.sum(f(predicts*y))
    num_predicts = y.shape[1]
    acc = num_correct_predicts / num_predicts

    return predicts, acc

        



In [261]:
dt_model = DecisionTree(max_depth=10)
dt_model.fit(X_train,y_train)

X shape: (3878, 300)
y shape: (1, 3878)
ids shape: (3878,)


In [262]:
predicts, acc = dt_model.test(X_test, y_test)

In [263]:
acc

0.8244392884764115

# VII. Test models with banknote authentication dataset

In [264]:
banknote_dataset = pd.read_csv('drive/MyDrive/desktop/ADA/extra/data_banknote_authentication.txt', delimiter = ",", header=None)

In [265]:
banknote_dataset

,0,1,2,3,4
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


In [266]:
banknote_X = banknote_dataset.iloc[:,:-1].to_numpy()
banknote_y = banknote_dataset.iloc[:,-1].to_numpy()
banknote_y = np.where(banknote_y == 0, -1, banknote_y)

In [267]:
banknote_X_train, banknote_X_test, banknote_y_train, banknote_y_test = train_test_split(banknote_X, banknote_y)
banknote_X_train = banknote_X_train.T
banknote_X_test = banknote_X_test.T
banknote_y_train = banknote_y_train.reshape(1, banknote_y_train.shape[0])
banknote_y_test = banknote_y_test.reshape(1, banknote_y_test.shape[0])

In [268]:
print(banknote_X_train.shape)
print(banknote_y_train.shape)
print(banknote_X_test.shape)
print(banknote_y_test.shape)

(4, 1029)
(1, 1029)
(4, 343)
(1, 343)


In [269]:
dt_model = NewDecisionTree(max_depth=10)
dt_model.fit(banknote_X_train, banknote_y_train)

X shape: (1029, 4)
y shape: (1, 1029)
ids shape: (1029,)


In [270]:
predicts, acc = dt_model.test(banknote_X_test, banknote_y_test)

In [271]:
acc

0.9795918367346939

In [272]:
model = SVM()
model.fit(banknote_X_train, banknote_y_train)

Acc: 0.1467444120505345
Acc: 0.9931972789115646
Acc: 0.9931972789115646
Acc: 0.9941690962099126
Acc: 0.9931972789115646
Acc: 0.9951409135082604
Acc: 0.9922254616132167
Acc: 0.9922254616132167
Acc: 0.9922254616132167
Acc: 0.9922254616132167


In [273]:
predicts, acc = model.predict(banknote_X_test, banknote_y_test)

In [274]:
acc

0.9795918367346939

In [275]:
model = kNN()
model.fit(banknote_X_train, banknote_y_train)
predicts, acc = model.predict(banknote_X_test, banknote_y_test)

In [276]:
acc

1.0